In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time
from reconstruct_functions import *

In [2]:
hop_length = 2
win_length = 8 ###at least 4*hop_length, even number
channels = win_length//2+1

def load_wav_to_torch(full_path):
    sampling_rate, data = read(full_path)
    data = data.astype(np.float64)
    # data = np.round(data,decimals=6)
    return torch.DoubleTensor(data), sampling_rate

audio_origin, sampling_rate = load_wav_to_torch('demo.wav')
audio_origin = audio_origin[None,:]

# audio_origin = torch.rand((1,wav_length), dtype=torch.float64)

print(audio_origin.shape)
print(audio_origin[0,:10])

torch.Size([1, 98560])
tensor([ 0.0380,  0.0185, -0.0360, -0.0357,  0.0195,  0.0406, -0.0009, -0.0261,
        -0.0027,  0.0140], dtype=torch.float64)


In [3]:
# stft_fn = stft.STFT(filter_length=4, hop_length=1, win_length=4,
#                     window='hann')

stft_fn = stft.STFT(filter_length=win_length, hop_length=hop_length, win_length=win_length,
                    window=None)


def compare(a,b):
    return torch.mean(torch.abs(a-b)), torch.mean((a-b)*(a-b))

def compare_L1(ori,gen):
    return torch.mean(torch.abs(ori-gen)/torch.abs(ori))


def compare_L2(a,b):
    return torch.sum(torch.abs(a-b)), torch.sum((a-b)*(a-b))


In [4]:
magnitude, phase_origin = stft_fn.transform(audio_origin)

forward_basis = stft_fn.forward_basis
print(forward_basis.shape)
start_frame = 0
print(magnitude[0,:,start_frame])
magnitude    = magnitude[:,:,2:-2]
print(magnitude.shape)


torch.Size([10, 1, 8])
tensor([0.0488, 0.0952, 0.1295, 0.0582, 0.0198], dtype=torch.float64)
torch.Size([1, 5, 49277])


In [5]:
ans = reconstruct_from_S_with_H_2(magnitude, stft_fn, hop_length=2, frame_number=None)


100%|██████████| 49274/49274 [23:14<00:00, 35.34it/s]  


In [6]:
if ans[0,0]*audio_origin[0,0]<0:
    ans[:,:] = -ans[:,:]

In [7]:

start_frame = 0
end_frame = 1900
T_start = start_frame * hop_length + win_length-hop_length
T_end = end_frame * hop_length + win_length-hop_length
# print(ans[0,T_start:T_end])
# print(audio_origin[0,T_start:T_end])
print(torch.mean(torch.abs(audio_origin[0,T_start:T_end] - ans[0,T_start:T_end])))

tensor(3.5160e-15, dtype=torch.float64)


In [8]:

print(ans[0,-20:])
print(audio_origin[0,-20:])
print(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))

[-1.26924727e-03 -1.14603038e-03 -8.32255930e-04 -1.07198174e-03
 -7.56122463e-04 -4.71334875e-04 -3.07979644e-04 -1.74949091e-04
 -2.47407472e-04  5.36974985e-05  8.55077815e-05 -4.05335304e-05
  1.85433921e-04  3.66384891e-04  1.64354744e-04  2.64009112e-04
  3.09011841e-04  8.89946241e-05  1.77230206e-04  3.73422517e-04]
tensor([-1.2692e-03, -1.1460e-03, -8.3226e-04, -1.0720e-03, -7.5612e-04,
        -4.7133e-04, -3.0798e-04, -1.7495e-04, -2.4741e-04,  5.3697e-05,
         8.5508e-05, -4.0534e-05,  1.8543e-04,  3.6638e-04,  1.6435e-04,
         2.6401e-04,  3.0901e-04,  8.8995e-05,  1.7723e-04,  3.7342e-04],
       dtype=torch.float64)
tensor(7.0086e-14, dtype=torch.float64)
